In [1]:
import sire as sr

In [2]:
url = "https://github.com/OpenBioSim/posts/raw/main/sire/001_ten_minute"

In [3]:
mols = sr.load(sr.expand(url, "SYSTEM.top", "SYSTEM.crd"))

Unzipping './SYSTEM.top.bz2'...
Unzipping './SYSTEM.crd.bz2'...


In [4]:
mols

System( name=BioSimSpace Syst num_molecules=18575 num_residues=18914 num_atoms=60695 )

In [ ]:
mols["water"]

In [ ]:
mols["protein"]

In [ ]:
mols["not (protein or water)"]

In [ ]:
ligand = mols["count(atoms) > 1 and not (protein or water)"]
ligand

In [ ]:
ligand.view()

In [ ]:
ligand.energy()

In [ ]:
ligand.energy().components()

In [ ]:
sr.search.set_token("ligand", "count(atoms) > 1 and not (protein or water)")

In [ ]:
residues = mols["(residues within 3 of ligand) and protein"]

In [ ]:
residues.molecules()

In [ ]:
residues.view()

In [ ]:
data = {"residue": [], "component": [], "energy": []}

for residue, energy in residues.apply(lambda res: (res, res.energy(ligand))):
    resid = f"{residue.name().value()}:{residue.number().value()}"
    
    for component in energy.components():
        data["residue"].append(resid)
        data["component"].append(component)
        data["energy"].append(energy[component].to(sr.units.kcal_per_mol))
    
    data["residue"].append(resid)
    data["component"].append("total")
    data["energy"].append(energy.to(sr.units.kcal_per_mol))

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df

In [ ]:
df[ df["component"] == "total" ].plot.bar(x="residue")

In [ ]:
df[ df["component"] == "coulomb" ].plot.bar(x="residue")

In [ ]:
mols["ligand or (resname LYS and resnum 33)"].view()

In [ ]:
closest = (1000 * sr.units.angstrom, None, None)

for atom0 in mols["resname LYS and resnum 33"].atoms():
    for atom1 in ligand:
        dist = sr.measure(atom0, atom1)
        if dist < closest[0]:
            closest = (dist, atom0, atom1)

In [ ]:
print(closest)

In [ ]:
closest[1].energy(closest[2]).components()